In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from numpy import hstack
import warnings
warnings.filterwarnings('ignore')
from sklearn.naive_bayes import MultinomialNB
import os
import glob
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline

In [2]:
path=os.getcwd()
print(path)

C:\Users\Udeshya Dixit


In [3]:
import zipfile as zf
positive_files = zf.ZipFile("positve.zip", 'r')
positive_files.extractall('directory to extract')
positive_files.close()

negative_files = zf.ZipFile("negative.zip", 'r')
negative_files.extractall('directory1 to extract')
negative_files.close()



KeyboardInterrupt: 

In [4]:
positive=os.listdir(r'directory to extract\positve')
negative=os.listdir(r'directory1 to extract\negative')

In [5]:
print(type(positive),len(negative))

<class 'list'> 8000


In [99]:
data_positive=pd.DataFrame(positive)
data_positive.columns=['file_name']

ones=pd.DataFrame([1]*10000)
ones.columns=['class']

positive_class=pd.concat([data_positive,ones],axis=1)
positive_class.head()

,file_name,class
0,0_9.txt,1
1,10000_8.txt,1
2,10002_7.txt,1
3,10004_8.txt,1
4,10005_7.txt,1


In [7]:
positive_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
file_name    10000 non-null object
class        10000 non-null int64
dtypes: int64(1), object(1)
memory usage: 156.3+ KB


In [100]:
data_negative=pd.DataFrame(negative,index=[10000+x for x in range(8000)])
data_negative.columns=['file_name']

zeroes=pd.DataFrame([0]*8000,index=[10000+x for x in range(8000)])
zeroes.columns=['class']

negative_class=pd.concat([data_negative,zeroes],axis=1)
negative_class.head()

,file_name,class
10000,10000_4.txt,0
10001,10001_4.txt,0
10002,10003_1.txt,0
10003,10004_3.txt,0
10004,10006_4.txt,0


In [101]:
negative_class.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 10000 to 17999
Data columns (total 2 columns):
file_name    8000 non-null object
class        8000 non-null int64
dtypes: int64(1), object(1)
memory usage: 187.5+ KB


In [103]:
data=pd.concat([positive_class,negative_class])
data.head()

,file_name,class
0,0_9.txt,1
1,10000_8.txt,1
2,10002_7.txt,1
3,10004_8.txt,1
4,10005_7.txt,1


In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18000 entries, 0 to 17999
Data columns (total 2 columns):
file_name    18000 non-null object
class        18000 non-null int64
dtypes: int64(1), object(1)
memory usage: 421.9+ KB


In [105]:
data.describe()

,class
count,18000.000000
mean,0.555556
std,0.496918
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [106]:
data_folder = r'/Users/Udeshya Dixit\Videos/Machine Learning/text_classification_project/negative'
data['Script'] = [open(data_folder + os.sep + file,encoding="utf8").read() for file in data['file_name']]


In [107]:
data.head()

,file_name,class,Script
0,0_9.txt,1,Bromwell High is a cartoon comedy. It ran at t...
1,10000_8.txt,1,Homelessness (or Houselessness as George Carli...
2,10002_7.txt,1,This is easily the most underrated film inn th...
3,10004_8.txt,1,"This isn't the comedic Robin Williams, nor is ..."
4,10005_7.txt,1,Yes its an art... to successfully make a slow ...


In [108]:
print(data.isnull().sum())

file_name    0
class        0
Script       0
dtype: int64


In [109]:
data['Script'][10000]

"Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan) & her son. The luxury jetliner takes off as planned but mid-air the plane is hi-jacked by the co-pilot Chambers (Robert Foxworth) & his two accomplice's Banker (Monte Markham) & Wilson (Michael Pataki) who knock the passengers & crew out with sleeping gas, they plan to steal the valuable cargo & land on a disused plane strip on an isolated island but while making his descent Chambers almost hits an oil rig in the Ocean & loses control of the plane sending it crashing into the sea where it sinks to the bottom right bang in the middle of the Bermuda Triangle. With air in short supply, water leaking in & having flown over 200 miles off course the problems mount for 

In [110]:
b='lemon'
for i in range(data.shape[0]):
    a=data['Script'][i]
    if type(a) is not type(b):
        a=a.to_string()
    a=a.split()
    a=np.array(a)
    l=len(a)
    c=[]
    for j in range(l):
        k=len(a[j])>=3
        c.append(k)
    a=a[c]
    x=" ".join(a)
    #a=[x]
    data['Script'][i]=x
                              


In [112]:
y=data['class']
x=data['Script']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,stratify=y,random_state=100)
x_train_left,x_test_cv,y_train_left,y_test_cv=train_test_split(x_train,y_train,stratify=y_train,train_size=0.8,random_state=50)

In [113]:
def diversity(data):
    l=list(data)
    result=[]
    for i in range(len(l)):
        if l[i]==0:
            result.append(0)
    percent0=len(result)/len(l)*100
    length=len(result)
    return length,percent0

In [114]:
diversity(y_train_left)

(5120, 44.44444444444444)

In [115]:
diversity(y_test)

(1600, 44.44444444444444)

In [116]:
diversity(y_test_cv)

(1280, 44.44444444444444)

In [117]:
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer(stop_words='english')
x_train_transformed=vec.fit_transform(x_train_left)
x_cv_transformed=vec.transform(x_test_cv)
x_test_transformed=vec.transform(x_test)

In [118]:
print(x_train_transformed.shape,x_cv_transformed.shape,x_test_transformed.shape)

(11520, 54465) (2880, 54465) (3600, 54465)


In [119]:
#using naive_bayes

a=[10**x for x in range(-5,1)]
for i in a:
    mnb=MultinomialNB(alpha=i)
    mnb.fit(x_train_transformed,y_train_left)
    y_predict=mnb.predict(x_cv_transformed)
    print('for alpha=',i,'accuracy=',accuracy_score(y_test_cv,y_predict))
    

for alpha= 1e-05 accuracy= 0.8048611111111111
for alpha= 0.0001 accuracy= 0.8149305555555556
for alpha= 0.001 accuracy= 0.8256944444444444
for alpha= 0.01 accuracy= 0.840625
for alpha= 0.1 accuracy= 0.8552083333333333
for alpha= 1 accuracy= 0.8607638888888889


In [120]:
#testing for overfitting on best alpha
mnb=MultinomialNB(alpha=1)
mnb.fit(x_train_transformed,y_train_left)

y_predict_train=mnb.predict(x_train_transformed)
print('accuracy on training set for best alpha=',accuracy_score(y_train_left,y_predict_train))

y_predict_cv=mnb.predict(x_cv_transformed)
print('accuracy on CV set for best alpha=',accuracy_score(y_test_cv,y_predict_cv))

y_predict_test=mnb.predict(x_test_transformed)
print('accuracy on test set for best alpha=',accuracy_score(y_test,y_predict_test))


accuracy on training set for best alpha= 0.9321180555555556
accuracy on CV set for best alpha= 0.8607638888888889
accuracy on test set for best alpha= 0.8619444444444444


In [121]:
#using SVM
a=[10**x for x in range(-5,1)]
for i in a:
    sgd=SGDClassifier(penalty='l2',loss='hinge',alpha=i,class_weight='balanced')
    sgd.fit(x_train_transformed,y_train_left)
    y1_predict=sgd.predict(x_cv_transformed)
    print('for alpha=',i,'accuracy=',accuracy_score(y_test_cv,y1_predict))

for alpha= 1e-05 accuracy= 0.8597222222222223
for alpha= 0.0001 accuracy= 0.8635416666666667
for alpha= 0.001 accuracy= 0.8659722222222223
for alpha= 0.01 accuracy= 0.8739583333333333
for alpha= 0.1 accuracy= 0.853125
for alpha= 1 accuracy= 0.7513888888888889


In [122]:
#testing for overfitting on best alpha
sgd=SGDClassifier(penalty='l2',loss='hinge',alpha=.01,class_weight='balanced')
sgd.fit(x_train_transformed,y_train_left)

y1_predict_train=sgd.predict(x_train_transformed)
print('accuracy on training set for best alpha=',accuracy_score(y_train_left,y1_predict_train))

y1_predict_cv=sgd.predict(x_cv_transformed)
print('accuracy on CV set for best alpha=',accuracy_score(y_test_cv,y1_predict_cv))

y1_predict_test=sgd.predict(x_test_transformed)
print('accuracy on test set for best alpha=',accuracy_score(y_test,y1_predict_test))


accuracy on training set for best alpha= 0.9497395833333333
accuracy on CV set for best alpha= 0.8788194444444445
accuracy on test set for best alpha= 0.885


In [123]:
#using logistic regression
a=[10**x for x in range(-5,1)]
for i in a:
    sgd1=SGDClassifier(penalty='l2',loss='log',alpha=i,class_weight='balanced')
    sgd1.fit(x_train_transformed,y_train_left)
    y2_predict=sgd1.predict(x_cv_transformed)
    print('for alpha=',i,'accuracy=',accuracy_score(y_test_cv,y2_predict))

for alpha= 1e-05 accuracy= 0.8652777777777778
for alpha= 0.0001 accuracy= 0.86875
for alpha= 0.001 accuracy= 0.8722222222222222
for alpha= 0.01 accuracy= 0.8611111111111112
for alpha= 0.1 accuracy= 0.8295138888888889
for alpha= 1 accuracy= 0.7454861111111111


In [124]:
#testing for overfitting on best alpha
sgd1=SGDClassifier(penalty='l2',loss='hinge',alpha=.001,class_weight='balanced')
sgd1.fit(x_train_transformed,y_train_left)

y2_predict_train=sgd1.predict(x_train_transformed)
print('accuracy on training set for best alpha=',accuracy_score(y_train_left,y2_predict_train))

y2_predict_cv=sgd1.predict(x_cv_transformed)
print('accuracy on CV set for best alpha=',accuracy_score(y_test_cv,y2_predict_cv))

y2_predict_test=sgd1.predict(x_test_transformed)
print('accuracy on test set for best alpha=',accuracy_score(y_test,y2_predict_test))

accuracy on training set for best alpha= 0.9952256944444444
accuracy on CV set for best alpha= 0.8694444444444445
accuracy on test set for best alpha= 0.8708333333333333


In [42]:
vec.vocabulary_

{'comedy': 9817,
 'movie': 32186,
 'boasts': 5783,
 'marvelous': 30060,
 'soundtrack': 45195,
 'cover': 11153,
 'know': 27054,
 'dealing': 12315,
 'notch': 33534,
 'rented': 40062,
 'friends': 19281,
 'expecting': 17058,
 'chuckles': 8947,
 'overall': 34677,
 'laughs': 27731,
 'making': 29524,
 'fun': 19446,
 'couldn': 11056,
 'chosen': 8871,
 'worse': 53760,
 'br': 6284,
 'alright': 2043,
 'changes': 8357,
 'painful': 34995,
 'physical': 36263,
 'gags': 19602,
 'poorly': 37011,
 'performed': 35869,
 'placed': 36552,
 'fat': 17660,
 'kid': 26754,
 'want': 52618,
 'kill': 26796,
 'bless': 5564,
 'trying': 50041,
 'scene': 42318,
 'like': 28306,
 'puppy': 38354,
 'begging': 4737,
 'love': 28896,
 'pulled': 38271,
 'bearable': 4580,
 'funny': 19486,
 'jokes': 26043,
 'believe': 4827,
 'group': 21256,
 'boys': 6276,
 'steal': 46076,
 'parent': 35266,
 'porn': 37064,
 'movies': 32195,
 'turns': 50184,
 'gay': 19892,
 'best': 5065,
 'sum': 47032,
 'simply': 44071,
 'tell': 48256,
 'opening':

In [4]:
'2001'.isdigit()

True